In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import IPython.display as ipd
import librosa
import librosa.display

In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
audio_dataset_path="Data/genres_original"

In [5]:
metadata=pd.read_csv("Data/features_30_sec.csv")
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [6]:
def features_extractor(file):
    audio, sample_rate=librosa.load(file_name, res_type="kaiser_fast")
    mfccs_features=librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [7]:
metadata.drop(labels=552, axis=0 , inplace=True)

In [8]:
from tqdm import tqdm
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    try:
        final_class_labels=row.label
        file_name=os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))
        data=features_extractor(file_name)
        extracted_features.append([data,final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

553it [01:08,  8.94it/s]C:\Users\ASUS\AppData\Local\Temp\ipykernel_31764\4115935976.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate=librosa.load(file_name, res_type="kaiser_fast")
C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
555it [01:08, 11.34it/s]

Error: 


999it [01:57,  8.47it/s]


In [9]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()
                                   

,feature,class
0,"[-113.59882, 121.57067, -19.162262, 42.363937,...",blues
1,"[-207.52383, 123.98514, 8.94702, 35.86715, 2.9...",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855278, 1....",blues
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",blues


In [10]:
extracted_features_df['class'].value_counts()

class
blues        100
classical    100
country      100
disco        100
hiphop       100
metal        100
pop          100
reggae       100
rock         100
jazz          98
Name: count, dtype: int64

In [11]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [12]:
X.shape

(998, 40)

In [13]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [14]:
y.shape

(998, 10)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
X_train

array([[-1.04723763e+02,  8.77537231e+01, -3.32488594e+01, ...,
        -2.38248777e+00, -1.36347115e+00, -7.22123563e-01],
       [-2.59909851e+02,  1.23193169e+02, -6.39508581e+00, ...,
        -6.73697710e+00, -3.90829563e+00,  3.18117642e+00],
       [-1.15755066e+02,  6.70791245e+01,  1.88346887e+00, ...,
        -3.43661404e+00, -1.73870683e+00, -4.68739159e-02],
       ...,
       [-1.25020428e+01,  9.13173676e+01, -2.30759563e+01, ...,
        -4.04763985e+00, -1.77685583e+00, -1.75431299e+00],
       [-2.37930965e+01,  8.29835587e+01,  2.32049274e+00, ...,
         1.40550268e+00,  4.16220337e-01, -3.45979966e-02],
       [-9.63197021e+01,  9.09497147e+01, -3.22195396e+01, ...,
        -2.41483903e+00, -1.62698254e-01, -1.84749973e+00]], dtype=float32)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(798, 40)
(200, 40)
(798, 10)
(200, 10)


In [18]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [20]:
num_labels=y.shape[1]

In [21]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [23]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [24]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training time", duration)



Epoch 1/100
19/25 [=====================>........] - ETA: 0s - loss: 26.7899 - accuracy: 0.0921
Epoch 1: val_loss improved from inf to 4.08631, saving model to saved_models\audio_classification.hdf5
25/25 [==============================] - 3s 35ms/step - loss: 23.6430 - accuracy: 0.1015 - val_loss: 4.0863 - val_accuracy: 0.0700
Epoch 2/100
 1/25 [>.............................] - ETA: 0s - loss: 15.8797 - accuracy: 0.0625

C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/25 [====================>.........] - ETA: 0s - loss: 11.7192 - accuracy: 0.1128
Epoch 2: val_loss improved from 4.08631 to 2.37381, saving model to saved_models\audio_classification.hdf5
25/25 [==============================] - 0s 11ms/step - loss: 10.8000 - accuracy: 0.1065 - val_loss: 2.3738 - val_accuracy: 0.1600
Epoch 3/100
20/25 [=======================>......] - ETA: 0s - loss: 7.2650 - accuracy: 0.1359
Epoch 3: val_loss improved from 2.37381 to 2.23035, saving model to saved_models\audio_classification.hdf5
25/25 [==============================] - 0s 12ms/step - loss: 6.9274 - accuracy: 0.1328 - val_loss: 2.2304 - val_accuracy: 0.2450
Epoch 4/100
18/25 [====================>.........] - ETA: 0s - loss: 4.8819 - accuracy: 0.1406
Epoch 4: val_loss improved from 2.23035 to 2.21887, saving model to saved_models\audio_classification.hdf5
25/25 [==============================] - 0s 12ms/step - loss: 4.8236 - accuracy: 0.1278 - val_loss: 2.2189 - val_accuracy: 0.2100
Epoch 5/100
20

In [25]:
model.evaluate(X_test,y_test,verbose=0)

[1.4269911050796509, 0.5099999904632568]

In [26]:
X_test[1]

array([-75.35942   ,  83.35071   ,  10.695162  ,  14.4605    ,
        14.239106  ,   7.397377  ,   5.2808986 ,  -0.3932319 ,
         5.72108   ,   3.3658257 ,  -2.5226784 ,  -4.2161746 ,
        -1.8445053 ,  -5.9071975 ,  -1.2714547 ,  -3.7691813 ,
        -2.8537426 ,  -0.93226945,  -1.1161617 ,  -2.7156475 ,
        -4.133138  ,  -0.56480694,   2.5117862 ,   1.8615217 ,
         2.008656  ,   2.5816445 ,  -1.264946  ,  -4.4238324 ,
        -2.666795  ,   0.88555604,   1.2533294 ,  -2.900544  ,
        -4.5862637 ,  -2.5228503 ,  -0.8272398 ,   0.39746803,
        -0.5501506 ,  -2.2381675 ,  -0.31077942,  -2.1934414 ],
      dtype=float32)

In [27]:
np.argmax(model.predict(X_test), axis=-1)

7/7 [==============================] - 0s 3ms/step


array([0, 7, 0, 2, 6, 7, 5, 9, 8, 2, 6, 4, 1, 1, 5, 7, 6, 5, 5, 5, 8, 5,
       2, 0, 1, 7, 5, 0, 8, 6, 7, 4, 6, 8, 6, 6, 7, 5, 2, 7, 9, 9, 5, 3,
       0, 8, 3, 3, 2, 6, 5, 8, 6, 5, 0, 2, 3, 2, 3, 6, 3, 5, 9, 3, 4, 2,
       1, 4, 3, 1, 6, 3, 8, 9, 8, 9, 5, 4, 6, 3, 0, 6, 8, 8, 2, 5, 5, 5,
       8, 9, 3, 0, 3, 9, 6, 7, 7, 5, 2, 5, 5, 8, 4, 8, 1, 4, 7, 8, 1, 6,
       4, 4, 3, 8, 6, 2, 4, 6, 0, 5, 4, 5, 5, 0, 2, 7, 5, 8, 2, 3, 8, 9,
       5, 4, 1, 9, 5, 4, 9, 2, 0, 9, 6, 0, 3, 0, 3, 3, 4, 8, 6, 5, 7, 8,
       8, 9, 4, 4, 4, 8, 9, 8, 6, 4, 7, 1, 7, 5, 7, 5, 8, 6, 1, 7, 7, 1,
       4, 7, 2, 7, 5, 9, 6, 5, 8, 8, 5, 6, 0, 7, 6, 6, 1, 2, 5, 2, 6, 3,
       0, 5], dtype=int64)

In [28]:
filename="Data/genres_original/classical/classical.00001.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=np.argmax(model.predict(mfccs_scaled_features), axis=-1)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

[-324.09128     133.56267     -25.030157     24.55361     -11.93992
   17.658503     -5.5186734     1.7168901   -10.575926     -0.8605745
   -7.80522       3.5297625     5.0546646     1.3707236     3.0050094
   -3.1251519    -4.289579      0.59402454    2.877037      0.37455818
   -1.2144454     0.7690288     2.2079859     1.0528629     5.3391447
    2.526477      3.4166243    -1.2346429     6.6579533    11.365144
   -0.9949533   -11.721296     -6.2246456     4.2277317     7.017335
    5.0720463     2.523582     -5.4079676    -6.1680713     0.7645855 ]
[[-324.09128     133.56267     -25.030157     24.55361     -11.93992
    17.658503     -5.5186734     1.7168901   -10.575926     -0.8605745
    -7.80522       3.5297625     5.0546646     1.3707236     3.0050094
    -3.1251519    -4.289579      0.59402454    2.877037      0.37455818
    -1.2144454     0.7690288     2.2079859     1.0528629     5.3391447
     2.526477      3.4166243    -1.2346429     6.6579533    11.365144
    -0.9949533   

array(['classical'], dtype='<U9')

In [29]:
filename="Data/genres_original/hiphop/hiphop.00003.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=np.argmax(model.predict(mfccs_scaled_features),axis=-1)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

[-4.0301540e+01  8.0847237e+01 -2.5676889e+01  3.8443054e+01
 -1.1072825e+01  2.4899410e+01 -7.3732128e+00  2.6759275e+01
 -5.3060298e+00  1.6782677e+01 -3.5075712e+00  8.4589281e+00
 -5.6017284e+00  1.2050404e+01 -1.8917377e+00  1.0464018e+01
  6.4275295e-01  2.6245183e-01 -5.0987406e+00 -4.0196013e+00
 -2.4602270e+00 -2.3325288e+00 -6.7665046e-01 -1.6401160e+00
  1.9417481e+00 -5.4288489e-01 -1.4439516e-01 -1.1494575e+00
  3.1748803e+00 -1.1912193e-02 -1.1991701e+00 -4.0615096e+00
 -1.0737150e+00 -2.1080587e+00 -8.2540566e-01 -3.4200957e+00
 -1.9619328e+00 -1.4907260e+00 -1.3176049e+00 -3.2468905e+00]
[[-4.0301540e+01  8.0847237e+01 -2.5676889e+01  3.8443054e+01
  -1.1072825e+01  2.4899410e+01 -7.3732128e+00  2.6759275e+01
  -5.3060298e+00  1.6782677e+01 -3.5075712e+00  8.4589281e+00
  -5.6017284e+00  1.2050404e+01 -1.8917377e+00  1.0464018e+01
   6.4275295e-01  2.6245183e-01 -5.0987406e+00 -4.0196013e+00
  -2.4602270e+00 -2.3325288e+00 -6.7665046e-01 -1.6401160e+00
   1.9417481e+00 

array(['hiphop'], dtype='<U9')

In [32]:
import pickle
pickle.dump(model,open('model.pkl','wb'))